In [1]:
import psycopg2
conn = psycopg2.connect(dbname="dq", user="dq")
conn.autocommit = True
cur = conn.cursor()

In [ ]:
cur.execute("CREATE DATABASE crimes_db;")
conn.close()

In [2]:
conn = psycopg2.connect(dbname = "crimes_db", user="dq")
cur = conn.cursor()
cur.execute("CREATE SCHEMA crimes")

In [3]:
import csv
with open('boston.csv','r') as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)

In [4]:
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


A function to compute a Python set with all distinct values contained in that column

In [5]:
def get_col_set(csv_file, col_index):
    import csv
    #initiate python set so cannot contain duplicates
    values = set()
    with open(csv_file, 'r') as file:
        next(file)
        reader = csv.reader(file)
        for row in reader:
            values.add(row[col_index])
    return values

for i in range(len(col_headers)):
    values = get_col_set("boston.csv", i)
    print(col_headers[i], len(values))

incident_number 298329
offense_code 219
description 239
date 1177
day_of_the_week 7
lat 18177
long 18177


A table inside crime schema

In [6]:
# create enumerated datatype for day of the week
cur.execute("""
    CREATE TYPE weekday AS ENUM ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');
""")

# find max length of "description" column
descriptions = get_col_set("boston.csv", 2)
max_len = 0
for description in descriptions:
    max_len = max(max_len, len(description))
print(max_len)

58


In [7]:
# create the table
cur.execute('''
    CREATE TABLE crimes.boston_crimes (
        incident_number INTEGER PRIMARY KEY,
        offense_code INTEGER,
        description VARCHAR(100),
        date DATE,
        day_of_the_week weekday,
        lat decimal,
        long decimal
    );
        ''')

In [8]:
#load boston.csv into table
with open("boston.csv") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)

In [12]:
cur.execute("SELECT * FROM crimes.boston_crimes")
#check length to see if all is loaded in
print(len(cur.fetchall()))

298329


create 2 groups of users - readonly and readwrite

In [14]:
#first make sure there are no privileges inherited from the public group and on the public schema

cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
cur.execute("REVOKE ALL ON DATABASE crimes_db FROM public;")

In [15]:
#create 2 users groups
#readonly group first
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crimes_db TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")



In [16]:
#readwrite groupd
cur.execute("CREATE GROUP readwrite NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crimes_db TO readwrite;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")
cur.execute("GRANT INSERT, SELECT ON ALL TABLES IN SCHEMA crimes TO readwrite;;")

try create one user for each group

In [17]:
cur.execute("CREATE USER readonly_user WITH PASSWORD 'readok';")
cur.execute("GRANT readonly TO readonly_user;")

cur.execute("CREATE USER readwrite_user WITH PASSWORD 'readwriteok';")
cur.execute("GRANT readwrite TO readwrite_user;")

testing

In [18]:
conn.close()
